<a href="https://colab.research.google.com/github/Gagalin105/TAHRD2nd-PL/blob/main/HW2%EF%BC%9AAI%E8%BC%94%E5%8A%A9%E5%88%86%E6%9E%90%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E8%88%87%E5%B9%B3%E8%A1%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一）


*   輸入種類與數值
*   根據種類分類


*   計算總收入





# **標頭檔**


In [ ]:
import pandas as pd
from datetime import datetime
import re
import gspread
from google.colab import auth
from google.auth import default

# Authenticate and authorize gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize global variables
total_income = 0

import gradio as gr
def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://249d138a12683296ae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# **函式：輸入**

In [ ]:
def get_expense_input():
    """Gets expense details from the user."""
    date_input = input("請輸入日期 (YYYY-MM-DD): ")
    time_input = input("請輸入時間 (HH:MM): ")
    item_input = input("請輸入項目: ")
    amount_input_str = input("請輸入金額: ")
    category_input = input("請根據先前輸入的項目分類，並輸入對應的數字：1. 收入、2. 吃喝、3. 娛樂: ")

    try:
        amount_input = float(amount_input_str)
    except ValueError:
        print("無效的金額輸入。請輸入數字。")
        return None, None, None, None, None # Return None for all if amount is invalid

    return date_input, time_input, item_input, amount_input, category_input

# Modify the add_expense function to accept parameters
def add_expense(date_str, time_str, item, amount, category_input):
    try:
        # Validate and correct date format (YYYY-MM-DD)
        date_match = re.match(r'(\d{4})[=/-](\d{1,2})[=/-](\d{1,2})', date_str)
        if not date_match:
            print("日期格式錯誤：請使用 YYYY-MM-DD 格式。")
            return

        year, month, day = date_match.groups()
        # Attempt to correct month and day if they are single digits
        corrected_date_str = f"{year}-{int(month):02d}-{int(day):02d}"

        # Validate and correct time format (HH:MM)
        time_match = re.match(r'^(\d{1,2}):(\d{1,2})$', time_str)
        if not time_match:
            print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
            return

        hour, minute = time_match.groups()
        # Attempt to correct hour and minute if they are single digits and validate values
        try:
            corrected_time_str = f"{int(hour):02d}:{int(minute):02d}"
            # Further validation for hour and minute values
            if not (0 <= int(hour) <= 23 and 0 <= int(minute) <= 59):
                 print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。時或分超出有效範圍。")
                 return
        except ValueError:
             print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
             return


        # Combine corrected date and time strings and convert to datetime object
        datetime_str = f"{corrected_date_str} {corrected_time_str}"
        datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')


        # --- Category input and validation (moved from original add_expense) ---
        category_map = {'1': '收入', '2': '吃喝', '3': '娛樂'}

        if category_input not in category_map:
            print("無效的分類輸入。請輸入 1, 2, 或 3。")
            return

        category = category_map[category_input]
        # --- End of category validation ---


        # Create a new row as a list for gspread
        # Add the category to the list
        new_row_list = [datetime_obj.date().strftime('%Y-%m-%d'),
                        datetime_obj.time().strftime('%H:%M'),
                        item,
                        amount,
                        category] # Add the category here


        # Append the new row to the Google Sheet using gspread
        # Use the gc object from the previous cell
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1 # Assuming the data is in the first sheet

        worksheet.append_row(new_row_list)

        print("Row added to Google Sheet successfully!")

        # --- New code for updating total income ---
        global total_income
        if category == '收入':
            total_income += amount
        else:
            total_income -= amount

        print(f"目前的總收入為 {total_income} 元")
        # --- End of new code ---


        # Optional: Update the DataFrame in memory after writing to the sheet
        global df
        if 'df' in globals() and isinstance(df, pd.DataFrame):
            new_row_dict = {'日期': datetime_obj.date(),
                            '時間': datetime_obj.time(),
                            '項目': item,
                            '金額': amount,
                            '類型': category} # Add category to DataFrame dict
            # Use pd.concat instead of append for newer pandas version

    except ValueError as e:
        print(f"Error adding row: {e}")
        print("請確保日期和時間格式正確。")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example of how to use the functions
# date_input, time_input, item_input, amount_input, category_input = get_expense_input()
# if all([date_input, time_input, item_input, amount_input is not None, category_input]): # Check if all inputs are valid
#     add_expense(date_input, time_input, item_input, amount_input, category_input)

# **函式：menu**

In [ ]:
def display_menu():
    """Displays the main menu options to the user."""
    print("\n請選擇功能：")
    print("1. 新增支出/收入")
    print("2. 顯示所有記帳項目 (依時間排序)")
    print("3. 顯示財務總結")
    print("q. 退出")

# **函式：顯示儲存資料**

# **顯示時依時間排序**


In [ ]:
def display_all_items():
    """Reads, sorts, and displays all items from the Google Sheet."""
    # Access the Google Sheet using the gc object and the sheet URL
    sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
    worksheet = sheet.sheet1

    # Read all data from the worksheet
    data = worksheet.get_all_values()

    if len(data) > 1:
        df = pd.DataFrame(data[1:], columns=data[0])

        # Ensure '金額' column is numeric, coercing errors to NaN
        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df.dropna(subset=['金額'], inplace=True)

        # Ensure '日期' and '時間' columns exist
        if '日期' in df.columns and '時間' in df.columns:
             # Combine '日期' and '時間' into a single datetime column
             df['時間戳記'] = df['日期'] + ' ' + df['時間']
             # Convert the '時間戳記' column to datetime objects, handling potential errors
             df['時間戳記'] = pd.to_datetime(df['時間戳記'], errors='coerce')
             df.dropna(subset=['時間戳記'], inplace=True) # Drop rows with invalid datetime

             # Sort the DataFrame by the new '時間戳記' column in ascending order
             df = df.sort_values(by='時間戳記', ascending=True)

             # Return the DataFrame instead of displaying it here
             return df[['日期', '時間', '項目', '金額', '類型']]
        else:
             print("Google Sheet 中缺少 '日期' 或 '時間' 欄位，無法依時間排序顯示。")
             return df # Return unsorted if sorting not possible


    else:
        print("Google Sheet 是空的，沒有記帳項目。")
        return pd.DataFrame(columns=['日期', '時間', '項目', '金額', '類型']) # Return empty DataFrame

# **函式：財務總結**

In [ ]:
def display_financial_summary():
    """Reads data and returns the financial summary as a string."""
    try:
        # Access the Google Sheet and read data (similar to display_all_items)
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()

        summary_string = ""

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric and '類型' column exists
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)
            if '類型' not in df.columns:
                df['類型'] = '未知' # Default category if column is missing

            # Calculate total income and expenses
            total_income = df[df['類型'] == '收入']['金額'].sum()
            total_expense = df[df['類型'] != '收入']['金額'].sum()
            total_balance = total_income - total_expense

            summary_string += "財務總結:\n"
            summary_string += f"總收入: {total_income:.2f} 元\n"
            summary_string += f"總支出: {total_expense:.2f} 元\n"
            summary_string += f"總餘額: {total_balance:.2f} 元"
        else:
            summary_string += "Google Sheet 是空的，沒有記帳項目可供計算。\n"
            summary_string += "財務總結:\n"
            summary_string += "總收入: 0.00 元\n"
            summary_string += "總支出: 0.00 元\n"
            summary_string += "總餘額: 0.00 元"

        return summary_string

    except Exception as e:
        return f"計算或顯示財務總結時發生錯誤: {e}"

# **主迴圈**

In [ ]:
# Define the display_menu function here as it's called in this cell
def display_menu():
    """Displays the main menu options to the user."""
    print("\n請選擇功能：")
    print("1. 新增支出/收入")
    print("2. 顯示所有記帳項目 (依時間排序)")
    print("3. 顯示財務總結")
    print("q. 退出")

while True:
    display_menu()
    choice = input("請輸入您的選擇: ").lower()

    if choice == '1':
        date_input, time_input, item_input, amount_input, category_input = get_expense_input()
        if all([date_input, time_input, item_input, amount_input is not None, category_input]):
            add_expense(date_input, time_input, item_input, amount_input, category_input)
        else:
            print("輸入無效，請重試。")
    elif choice == '2':
        # Call display_all_items and handle the returned DataFrame
        df_items = display_all_items()
        if not df_items.empty:
            print("\n所有記帳項目 (依時間排序):")
            display(df_items) # Use display() for better formatting in Colab
        else:
            print("Google Sheet 是空的，沒有記帳項目。")
    elif choice == '3':
        # Call display_financial_summary and print the returned summary string
        summary_string = display_financial_summary()
        print("\n" + summary_string) # Add a newline before printing the summary
    elif choice == 'q':
        print("程式結束。")
        break
    else:
        print("無效的選擇，請重新輸入。")


請選擇功能：
1. 新增支出/收入
2. 顯示所有記帳項目 (依時間排序)
3. 顯示財務總結
q. 退出
請輸入您的選擇: 1
請輸入日期 (YYYY-MM-DD): 2025-09-18
請輸入時間 (HH:MM): 18:09
請輸入項目: 柚子
請輸入金額: 80
請根據先前輸入的項目分類，並輸入對應的數字：1. 收入、2. 吃喝、3. 娛樂: 2
Row added to Google Sheet successfully!
目前的總收入為 -80.0 元

請選擇功能：
1. 新增支出/收入
2. 顯示所有記帳項目 (依時間排序)
3. 顯示財務總結
q. 退出
請輸入您的選擇: q
程式結束。


In [ ]:
import matplotlib.pyplot as plt
import io
import base64

def create_pie_chart(expense_by_category):
    """Creates a pie chart from the expense by category data."""
    if not expense_by_category:
        # Return None or handle empty data case
        return None

    labels = expense_by_category.keys()
    sizes = expense_by_category.values()
    colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen'] # Example colors

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Save the plot to a bytes object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig1) # Close the plot to free up memory

    return buf

# Modify display_financial_summary_ui to also return the plot
def display_financial_summary_ui():
    """Calls the financial summary function and prepares data and plot for UI display."""
    summary_data = display_financial_summary()

    # Format the summary data into a string for the Textbox output
    summary_string = f"財務總結:\n"
    summary_string += f"總收入: {summary_data['total_income']:.2f} 元\n"
    summary_string += f"總支出: {summary_data['total_expense']:.2f} 元\n"
    summary_string += f"總餘額: {summary_data['total_balance']:.2f} 元\n\n"

    summary_string += "各分類支出:\n"
    if summary_data['expense_by_category']:
        for category, amount in summary_data['expense_by_category'].items():
            summary_string += f"{category}: {amount:.2f} 元\n"
    else:
        summary_string += "無分類支出資料。"

    # Create the pie chart
    pie_chart = create_pie_chart(summary_data['expense_by_category'])

    # Return both the summary string and the pie chart (BytesIO object)
    return summary_string, pie_chart

In [ ]:
# Import the necessary Gradio library (already imported in cell 38778d21, but good practice to include where used)
import gradio as gr

# Define a function that will be called by Gradio to handle adding an expense
# This function will take inputs from the Gradio interface and call your add_expense function
def add_expense_ui(date_str, time_str, item, amount_str, category_input):
    try:
        amount = float(amount_str)
        # Call the existing add_expense function
        add_expense(date_str, time_str, item, amount, category_input)
        return "新增記帳項目成功！"
    except ValueError:
        return "金額輸入無效。請輸入數字。"
    except Exception as e:
        return f"新增記帳項目時發生錯誤: {e}"

# Define a function to display all items
def display_all_items_ui():
    # Call the existing display_all_items function which now returns the DataFrame
    df_items = display_all_items()
    # Return the DataFrame for Gradio to display
    return df_items

# Define a function to display the financial summary
def display_financial_summary_ui():
     # Call the existing display_financial_summary function
    display_financial_summary()
    # Similar to display_all_items_ui, assuming display_financial_summary handles output.
    return "已在控制台顯示財務總結。" # Return a simple message for the UI

# Create the Gradio Interface
# We will create separate interfaces or a tabbed interface for different functionalities later if needed.
# Let's start with the 'Add Expense' interface.

add_expense_interface = gr.Interface(
    fn=add_expense_ui, # The function to call when the interface is used
    inputs=[
        gr.Textbox(label="日期 (YYYY-MM-DD)"),
        gr.Textbox(label="時間 (HH:MM)"),
        gr.Textbox(label="項目"),
        gr.Textbox(label="金額"),
        gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂") # Use Radio for categories
    ],
    outputs="text", # The output will be text indicating success or failure
    title="新增支出/收入",
    description="請輸入您的記帳項目詳細資訊。"
)

# You can create separate interfaces for other functions
display_items_interface = gr.Interface(
    fn=display_all_items_ui,
    inputs=None, # No inputs needed to display all items
    # Change the output to gr.DataFrame to display the returned DataFrame
    outputs=gr.DataFrame(label="所有記帳項目 (依時間排序)"),
    title="顯示所有記帳項目",
    description="點擊按鈕顯示所有記帳項目 (依時間排序)。"
)

financial_summary_interface = gr.Interface(
    fn=display_financial_summary_ui,
    inputs=None, # No inputs needed
    outputs="text", # Output a message
    title="顯示財務總結",
    description="點擊按鈕顯示財務總結 (將顯示在控制台)。"
)


# To present these as a single application with tabs, we can use gr.TabbedInterface or gr.Blocks
# Let's use gr.TabbedInterface for simplicity
tabbed_interface = gr.TabbedInterface(
    [add_expense_interface, display_items_interface, financial_summary_interface],
    ["新增支出/收入", "顯示所有記帳項目", "顯示財務總結"]
)


# Launch the Gradio app
tabbed_interface.launch(debug=True) # Set debug=True for more detailed error messages

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3a82a72ff04fcae4db.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Task
Modify the Gradio interface to display the accounting items table automatically when the "Display All Items" tab is selected, instead of requiring a button click.

## 修改 `display all items ui` 函式

### Subtask:
確認 `display_all_items_ui` 函式正確地讀取資料並返回 DataFrame。


**Reasoning**:
Review the `display_all_items_ui` and `display_all_items` functions to confirm that `display_all_items` reads data and returns a DataFrame and that `display_all_items_ui` correctly returns this DataFrame.



In [ ]:
# Reviewing the display_all_items function
# It connects to Google Sheet, reads data, creates a DataFrame,
# processes '金額', '日期', and '時間' columns, sorts by '時間戳記',
# and returns a DataFrame with selected columns. This seems correct.

# Reviewing the display_all_items_ui function
# It calls display_all_items() and returns the result. This also seems correct.

# No changes are needed to these functions for this specific subtask as they already
# perform the required actions of reading data and returning a DataFrame.
print("Reviewed display_all_items and display_all_items_ui functions. They are set up to read data and return a DataFrame as required.")

## 轉換為 `gr.blocks`

### Subtask:
將目前的 `gr.TabbedInterface` 結構轉換為使用 `gr.Blocks` 來構建分頁介面。


**Reasoning**:
The subtask requires converting the Gradio interface from `gr.TabbedInterface` to `gr.Blocks`. This involves removing the old interface code and creating a new structure using `gr.Blocks`, `gr.Tabs`, and `gr.TabItem` for each functionality.



In [ ]:
# Remove the existing gr.TabbedInterface creation and launch code
# This part is commented out as we are replacing it with gr.Blocks
# tabbed_interface = gr.TabbedInterface(
#     [add_expense_interface, display_items_interface, financial_summary_interface],
#     ["新增支出/收入", "顯示所有記帳項目", "顯示財務總結"]
# )
# tabbed_interface.launch(debug=True)

# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入"):
            # Place the components for adding expense here
            # These components are based on the original add_expense_interface inputs
            gr.Textbox(label="日期 (YYYY-MM-DD)"),
            gr.Textbox(label="時間 (HH:MM)"),
            gr.Textbox(label="項目"),
            gr.Textbox(label="金額"),
            gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂"),
            gr.Button("新增項目") # Add a button to trigger the add_expense_ui function

        # Create a TabItem for displaying all items
        with gr.TabItem("顯示所有記帳項目"):
            # Place the components for displaying all items here
            # The display_items_interface didn't have inputs, only output
            # We will add a button here to trigger the display for now
            gr.Button("顯示所有項目")
            # The output will be a DataFrame, but we are just structuring now
            gr.DataFrame(label="所有記帳項目 (依時間排序)") # Placeholder for the output DataFrame


        # Create a TabItem for displaying financial summary
        with gr.TabItem("顯示財務總結"):
             # Place the components for displaying financial summary here
             # financial_summary_interface had no inputs, only output
             gr.Button("顯示財務總結")
             gr.Textbox(label="財務總結") # Placeholder for the output text

# Launch the Blocks app (optional for this step, but good for testing structure)
# demo_blocks.launch(debug=True)

## 定義 gradio 元件

### Subtask:
在 `gr.Blocks` 中明確定義每個分頁所需的 Gradio 元件，包括用於顯示表格的 `gr.DataFrame`。


**Reasoning**:
Based on the instructions and the previous step, I need to ensure that the `gr.Blocks` structure correctly defines all necessary input and output components for each tab with appropriate labels, including buttons to trigger actions and `gr.DataFrame` for displaying the table. I will refine the components within each `gr.TabItem` as specified.



In [ ]:
# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入"):
            # Define inputs for adding expense
            add_date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
            add_time_input = gr.Textbox(label="時間 (HH:MM)")
            add_item_input = gr.Textbox(label="項目")
            add_amount_input = gr.Textbox(label="金額")
            add_category_input = gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂")
            # Define a button to trigger the add expense function
            add_expense_button = gr.Button("新增項目")
            # Define the output for the add expense function
            add_expense_output = gr.Textbox(label="結果")


        # Create a TabItem for displaying all items
        with gr.TabItem("顯示所有記帳項目"):
            # Define a button to trigger displaying all items
            display_items_button = gr.Button("顯示所有項目")
            # Define the output as a DataFrame to display the items
            display_items_output = gr.DataFrame(label="所有記帳項目 (依時間排序)")


        # Create a TabItem for displaying financial summary
        with gr.TabItem("顯示財務總結"):
             # Define a button to trigger displaying the summary
             display_summary_button = gr.Button("顯示財務總結")
             # Define the output as a Textbox to display the summary (which is printed to console by the function)
             # We'll keep a Textbox placeholder here for potential future direct display in UI
             display_summary_output = gr.Textbox(label="財務總結結果 (請查看控制台)")

# Note: We are not connecting the backend logic (functions) to these components yet.
# That will be done in a subsequent step. This step is purely about defining the UI components.

## 處理分頁切換事件

### Subtask:
使用 Gradio 的事件處理機制（例如 `gr.Tabs` 的 `select` 事件）來偵測使用者何時切換到「顯示所有記帳項目」分頁。


**Reasoning**:
Use gr.Blocks and gr.Tabs to structure the interface and then use the .select() method on the tabs object to trigger the display_all_items_ui function when the second tab (index 1) is selected.



In [ ]:
# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入", id=0):
            # Define inputs for adding expense
            add_date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
            add_time_input = gr.Textbox(label="時間 (HH:MM)")
            add_item_input = gr.Textbox(label="項目")
            add_amount_input = gr.Textbox(label="金額")
            add_category_input = gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂")
            # Define a button to trigger the add expense function
            add_expense_button = gr.Button("新增項目")
            # Define the output for the add expense function
            add_expense_output = gr.Textbox(label="結果")

            # Link the button click to the add_expense_ui function
            add_expense_button.click(
                fn=add_expense_ui,
                inputs=[add_date_input, add_time_input, add_item_input, add_amount_input, add_category_input],
                outputs=add_expense_output
            )


        # Create a TabItem for displaying all items
        with gr.TabItem("顯示所有記帳項目", id=1) as display_items_tab:
            # Define the output as a DataFrame to display the items
            display_items_output = gr.DataFrame(label="所有記帳項目 (依時間排序)")

        # Create a TabItem for displaying financial summary
        with gr.TabItem("顯示財務總結", id=2):
             # Define the output as a Textbox to display the summary (which is printed to console by the function)
             # We'll keep a Textbox placeholder here for potential future direct display in UI
             display_summary_output = gr.Textbox(label="財務總結結果 (請查看控制台)")
             # Add a button to trigger displaying the summary
             display_summary_button = gr.Button("顯示財務總結")
             display_summary_button.click(
                 fn=display_financial_summary_ui,
                 inputs=None,
                 outputs=display_summary_output
             )

    # Use the .select() method on the tabs object to trigger an action when a tab is selected
    # The index for "顯示所有記帳項目" is 1 (0-indexed)
    tabs.select(
        fn=display_all_items_ui,
        inputs=None, # No inputs needed for display_all_items_ui
        outputs=display_items_output, # The output DataFrame will be updated
        queue=False # Setting queue to False for immediate execution on select
    )


# Launch the Blocks app
demo_blocks.launch(debug=True)

**Reasoning**:
The traceback indicates that the `gr.Tabs.select()` method does not accept the `every` parameter. I need to remove this parameter from the `.select()` call.



In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import io
import base64
import pandas as pd
from datetime import datetime
import re
import gspread
from google.colab import auth
from google.auth import default
import matplotlib.font_manager as fm # Import font_manager

# Authenticate and authorize gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize global variables (if needed, though add_expense now uses return values)
# total_income = 0 # This global variable is not used in the current functions


# Define the existing add_expense function (needed for add_expense_ui)
def add_expense(date_str, time_str, item, amount, category_input):
    try:
        # Validate and correct date format (YYYY-MM-DD)
        date_match = re.match(r'(\d{4})[=/-](\d{1,2})[=/-](\d{1,2})', date_str)
        if not date_match:
            print("日期格式錯誤：請使用 YYYY-MM-DD 格式。")
            return "新增失敗：日期格式錯誤。" # Return error message

        year, month, day = date_match.groups()
        # Attempt to correct month and day if they are single digits
        corrected_date_str = f"{int(year):04d}-{int(month):02d}-{int(day):02d}"

        # Validate and correct time format (HH:MM)
        time_match = re.match(r'^(\d{1,2}):(\d{1,2})$', time_str)
        if not time_match:
            print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
            return "新增失敗：時間格式錯誤。" # Return error message

        hour, minute = time_match.groups()
        # Attempt to correct hour and minute if they are single digits and validate values
        try:
            corrected_time_str = f"{int(hour):02d}:{int(minute):02d}"
            # Further validation for hour and minute values
            if not (0 <= int(hour) <= 23 and 0 <= int(minute) <= 59):
                 print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。時或分超出有效範圍。")
                 return "新增失敗：時間超出有效範圍。" # Return error message
        except ValueError:
             print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
             return "新增失敗：時間格式錯誤。" # Return error message


        # Combine corrected date and time strings and convert to datetime object
        datetime_str = f"{corrected_date_str} {corrected_time_str}"
        datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')


        # --- Category input and validation ---
        category_map = {'1': '收入', '2': '吃喝', '3': '娛樂'}

        if category_input not in category_map:
            print("無效的分類輸入。請輸入 1, 2, 或 3。")
            return "新增失敗：無效的分類輸入。" # Return error message

        category = category_map[category_input]
        # --- End of category validation ---


        # Create a new row as a list for gspread
        new_row_list = [datetime_obj.date().strftime('%Y-%m-%d'),
                        datetime_obj.time().strftime('%H:%M'),
                        item,
                        amount,
                        category] # Add the category here


        # Append the new row to the Google Sheet using gspread
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1 # Assuming the data is in the first sheet

        worksheet.append_row(new_row_list)

        print("Row added to Google Sheet successfully!")
        return "新增記帳項目成功！" # Return success message

    except ValueError as e:
        print(f"Error adding row: {e}")
        return f"新增失敗：請確保日期和時間格式正確。\n錯誤訊息: {e}"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return f"新增記帳項目時發生意外錯誤: {e}"


# Define the existing display_all_items function here
def display_all_items():
    """Reads, sorts, and displays all items from the Google Sheet."""
    try:
        # Access the Google Sheet using the gc object and the sheet URL
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1

        # Read all data from the worksheet
        data = worksheet.get_all_values()

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric, coercing errors to NaN
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)

            # Ensure '日期' and '時間' columns exist
            if '日期' in df.columns and '時間' in df.columns:
                 # Combine '日期' and '時間' into a single datetime column
                 df['時間戳記'] = df['日期'] + ' ' + df['時間']
                 # Convert the '時間戳記' column to datetime objects, handling potential errors
                 df['時間戳記'] = pd.to_datetime(df['時間戳記'], errors='coerce')
                 df.dropna(subset=['時間戳記'], inplace=True) # Drop rows with invalid datetime

                 # Sort the DataFrame by the new '時間戳記' column in ascending order
                 df = df.sort_values(by='時間戳記', ascending=True)

                 # Return the DataFrame instead of displaying it here
                 # Ensure column order matches the expected output in UI
                 return df[['日期', '時間', '項目', '金額', '類型']]
            else:
                 print("Google Sheet 中缺少 '日期' 或 '時間' 欄位，無法依時間排序顯示。")
                 # Return the DataFrame without sorting if date/time columns are missing
                 # Ensure it still has the expected columns if they exist
                 cols_to_return = ['日期', '時間', '項目', '金額', '類型']
                 existing_cols = [col for col in cols_to_return if col in df.columns]
                 return df[existing_cols]


        else:
            print("Google Sheet 是空的，沒有記帳項目。")
            return pd.DataFrame(columns=['日期', '時間', '項目', '金額', '類型']) # Return empty DataFrame

    except Exception as e:
        print(f"讀取或顯示所有項目時發生錯誤: {e}")
        # Return an empty DataFrame and print error
        return pd.DataFrame(columns=['日期', '時間', '項目', '金額', '類型'])



# Define the existing display_financial_summary function here
# This function is used by display_financial_summary_ui
def display_financial_summary():
    """Reads data, calculates financial summary, and gets Gemini analysis."""
    try:
        # Access the Google Sheet and read data (similar to display_all_items)
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()

        summary_data = {
            "total_income": 0.00,
            "total_expense": 0.00,
            "total_balance": 0.00,
            "expense_by_category": {},
            "gemini_analysis": "目前沒有足夠的資料進行分析。" # Initialize with a default message
        }

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric and '類型' column exists
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)
            if '類型' not in df.columns:
                df['類型'] = '未知' # Default category if column is missing

            # Calculate total income and expenses
            total_income = df[df['類型'] == '收入']['金額'].sum()
            total_expense = df[df['類型'] != '收入']['金額'].sum()
            total_balance = total_income - total_expense

            # Calculate expenses by category (excluding '收入')
            expense_by_category = df[df['類型'] != '收入'].groupby('類型')['金額'].sum().to_dict()

            summary_data.update({
                "total_income": total_income,
                "total_expense": total_expense,
                "total_balance": total_balance,
                "expense_by_category": expense_by_category
            })

            # --- Add Gemini Analysis ---
            try:
                # Prepare data for Gemini analysis
                analysis_prompt = f"""
                請根據以下的財務資料進行分析並提供建議：

                總收入: {total_income:.2f} 元
                總支出: {total_expense:.2f} 元
                總餘額: {total_balance:.2f} 元

                各分類支出:
                """
                if expense_by_category:
                    for category, amount in expense_by_category.items():
                        analysis_prompt += f"- {category}: {amount:.2f} 元\n"
                else:
                    analysis_prompt += "無分類支出資料。\n"

                analysis_prompt += "\n請提供一個簡潔的財務分析和一些關於支出管理的建議 (請用繁體中文回答)。"

                # Ensure gemini_model is initialized (from a previous cell)
                if 'gemini_model' in globals() and gemini_model:
                    response = gemini_model.generate_content(analysis_prompt)
                    summary_data["gemini_analysis"] = response.text
                else:
                    summary_data["gemini_analysis"] = "Gemini 模型尚未初始化，無法進行分析。"


            except Exception as gemini_e:
                summary_data["gemini_analysis"] = f"呼叫 Gemini API 時發生錯誤: {gemini_e}"
            # --- End of Gemini Analysis ---


        return summary_data

    except Exception as e:
        print(f"計算財務總結時發生錯誤: {e}")
        # Return initial summary_data with error message in analysis
        summary_data["gemini_analysis"] = f"讀取或計算財務資料時發生錯誤: {e}"
        return summary_data

# Create a pie chart function (assuming this was in a previous cell and needed here)
def create_pie_chart(expense_by_category):
    """Creates a pie chart from the expense by category data."""
    if not expense_by_category:
        # Return None or handle empty data case
        return None

    labels = expense_by_category.keys()
    sizes = expense_by_category.values()
    colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen'] # Example colors

    # Configure matplotlib to use a font that supports Chinese characters
    # This assumes the 'ttf-wqy-zenhei' font is installed (done in a separate cell)
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Zen Hei'] # Set the font
    plt.rcParams['axes.unicode_minus'] = False # Avoid displaying minus sign as box


    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Save the plot to a bytes object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig1) # Close the plot to free up memory

    return buf

# Define the display_financial_summary_ui to call the updated function and handle its return
def display_financial_summary_ui():
    """Calls the financial summary function and prepares data and plot for UI display."""
    summary_data = display_financial_summary()

    # Format the summary data into a string for the Textbox output
    summary_string = f"財務總結:\n"
    summary_string += f"總收入: {summary_data['total_income']:.2f} 元\n"
    summary_string += f"總支出: {summary_data['total_expense']:.2f} 元\n"
    summary_string += f"總餘額: {summary_data['total_balance']:.2f} 元\n\n"

    summary_string += "各分類支出:\n"
    if summary_data['expense_by_category']:
        for category, amount in summary_data['expense_by_category'].items():
            summary_string += f"{category}: {amount:.2f} 元\n"
    else:
        summary_string += "無分類支出資料。"

    # Add Gemini Analysis to the summary string
    summary_string += "\n\nGemini 財務分析與建議:\n"
    summary_string += summary_data.get("gemini_analysis", "無分析結果。") # Use .get for safety


    # Create the pie chart
    pie_chart = create_pie_chart(summary_data['expense_by_category'])

    # Return the summary string, the pie chart (BytesIO object), and the Gemini analysis
    # Note: The order here must match the order of outputs in the .click() event
    return summary_string, pie_chart, summary_data.get("gemini_analysis", "無分析結果。")


# Define a function that will be called by Gradio to handle adding an expense
# This function will take inputs from the Gradio interface and call your add_expense function
def add_expense_ui(date_str, time_str, item, amount_str, category_input):
    try:
        amount = float(amount_str)
        # Call the existing add_expense function
        # add_expense now returns a string message
        result_message = add_expense(date_str, time_str, item, amount, category_input)
        return result_message
    except ValueError:
        return "新增失敗：金額輸入無效。請輸入數字。"
    except Exception as e:
        return f"新增記帳項目時發生錯誤: {e}"

# Define a function to display all items
def display_all_items_ui():
    # Call the existing display_all_items function which now returns the DataFrame
    df_items = display_all_items()
    # Return the DataFrame for Gradio to display
    return df_items



# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入", id=0):
            # Define inputs for adding expense
            add_date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
            add_time_input = gr.Textbox(label="時間 (HH:MM)")
            add_item_input = gr.Textbox(label="項目")
            add_amount_input = gr.Textbox(label="金額")
            add_category_input = gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂")
            # Define a button to trigger the add expense function
            add_expense_button = gr.Button("新增項目")
            # Define the output for the add expense function
            add_expense_output = gr.Textbox(label="結果")

            # Link the button click to the add_expense_ui function
            add_expense_button.click(
                fn=add_expense_ui,
                inputs=[add_date_input, add_time_input, add_item_input, add_amount_input, add_category_input],
                outputs=add_expense_output
            )


        # Create a TabItem for displaying all items
        with gr.TabItem("顯示所有記帳項目", id=1) as display_items_tab:
            # Define the output as a DataFrame to display the items
            display_items_output = gr.DataFrame(label="所有記帳項目 (依時間排序)")

        # Create a TabItem for displaying financial summary
        with gr.TabItem("顯示財務總結", id=2) as display_summary_tab:
             # Define the output as a Textbox to display the summary string
             display_summary_output = gr.Textbox(label="財務總結")
             # Add a component to display the plot
             summary_pie_chart = gr.Plot(label="分類支出圓餅圖")
             # Add a component to display the Gemini analysis
             gemini_analysis_output = gr.Textbox(label="Gemini 財務分析與建議")
             # Add a button to trigger displaying the summary and plot
             display_summary_button = gr.Button("顯示財務總結與圖表")

             # Link the button click to the display_financial_summary_ui function
             # The function now returns both the summary string, the plot, and the Gemini analysis
             display_summary_button.click(
                 fn=display_financial_summary_ui,
                 inputs=None, # No inputs needed
                 outputs=[display_summary_output, summary_pie_chart, gemini_analysis_output] # Output text, plot, and Gemini analysis
             )


    # Use the .select() method on the tabs object to trigger an action when a tab is selected
    # The index for "顯示所有記帳項目" is 1 (0-indexed)
    tabs.select(
        fn=display_all_items_ui,
        inputs=None, # No inputs needed for display_all_items_ui
        outputs=display_items_output, # The output DataFrame will be updated
        queue=False # Setting queue to False for immediate execution on select
    )

    # Use the .select() method on the tabs object to trigger an action when the summary tab is selected
    # The index for "顯示財務總結" is 2 (0-indexed)
    # We will connect the button click to update both the summary text and the plot in the next step.
    # For now, we just define the components.


# Launch the Blocks app
demo_blocks.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://470376ae01279a26f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://470376ae01279a26f0.gradio.live


# Task
將財務總結的結果顯示在gradio的介面，同時顯示一個三種分類消費的占比圓餅圖。

## 修改財務總結函式

### Subtask:
修改 `display_financial_summary` 函式，使其返回總收入、總支出、總餘額以及各分類的支出資料。


**Reasoning**:
The subtask requires modifying the `display_financial_summary` function to return financial data instead of printing it. I will update the function to calculate the required values and return them in a structured format.



In [ ]:
def display_financial_summary():
    """Reads data, calculates financial summary, and gets Gemini analysis."""
    try:
        # Access the Google Sheet and read data (similar to display_all_items)
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()

        summary_data = {
            "total_income": 0.00,
            "total_expense": 0.00,
            "total_balance": 0.00,
            "expense_by_category": {},
            "gemini_analysis": "目前沒有足夠的資料進行分析。" # Initialize with a default message
        }

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric and '類型' column exists
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)
            if '類型' not in df.columns:
                df['類型'] = '未知' # Default category if column is missing

            # Calculate total income and expenses
            total_income = df[df['類型'] == '收入']['金額'].sum()
            total_expense = df[df['類型'] != '收入']['金額'].sum()
            total_balance = total_income - total_expense

            # Calculate expenses by category (excluding '收入')
            expense_by_category = df[df['類型'] != '收入'].groupby('類型')['金額'].sum().to_dict()

            summary_data.update({
                "total_income": total_income,
                "total_expense": total_expense,
                "total_balance": total_balance,
                "expense_by_category": expense_by_category
            })

            # --- Add Gemini Analysis ---
            try:
                # Prepare data for Gemini analysis
                analysis_prompt = f"""
                請根據以下的財務資料進行分析並提供建議：

                總收入: {total_income:.2f} 元
                總支出: {total_expense:.2f} 元
                總餘額: {total_balance:.2f} 元

                各分類支出:
                """
                if expense_by_category:
                    for category, amount in expense_by_category.items():
                        analysis_prompt += f"- {category}: {amount:.2f} 元\n"
                else:
                    analysis_prompt += "無分類支出資料。\n"

                analysis_prompt += "\n請提供一個簡潔的財務分析和一些關於支出管理的建議 (請用繁體中文回答)。"

                # Ensure gemini_model is initialized (from a previous cell)
                if 'gemini_model' in globals() and gemini_model:
                    response = gemini_model.generate_content(analysis_prompt)
                    summary_data["gemini_analysis"] = response.text
                else:
                    summary_data["gemini_analysis"] = "Gemini 模型尚未初始化，無法進行分析。"


            except Exception as gemini_e:
                summary_data["gemini_analysis"] = f"呼叫 Gemini API 時發生錯誤: {gemini_e}"
            # --- End of Gemini Analysis ---


        return summary_data

    except Exception as e:
        print(f"計算財務總結時發生錯誤: {e}")
        # Return initial summary_data with error message in analysis
        summary_data["gemini_analysis"] = f"讀取或計算財務資料時發生錯誤: {e}"
        return summary_data

# Modify the display_financial_summary_ui to call the updated function and handle its return
def display_financial_summary_ui():
    """Calls the financial summary function and prepares data and plot for UI display."""
    summary_data = display_financial_summary()

    # Format the summary data into a string for the Textbox output
    summary_string = f"財務總結:\n"
    summary_string += f"總收入: {summary_data['total_income']:.2f} 元\n"
    summary_string += f"總支出: {summary_data['total_expense']:.2f} 元\n"
    summary_string += f"總餘額: {summary_data['total_balance']:.2f} 元\n\n"

    summary_string += "各分類支出:\n"
    if summary_data['expense_by_category']:
        for category, amount in summary_data['expense_by_category'].items():
            summary_string += f"{category}: {amount:.2f} 元\n"
    else:
        summary_string += "無分類支出資料。"

    # Add Gemini Analysis to the summary string
    summary_string += "\n\nGemini 財務分析與建議:\n"
    summary_string += summary_data.get("gemini_analysis", "無分析結果。") # Use .get for safety


    # Create the pie chart
    pie_chart = create_pie_chart(summary_data['expense_by_category'])

    # Return both the summary string and the pie chart (BytesIO object) and the Gemini analysis
    return summary_string, pie_chart

## 生成圓餅圖程式碼

### Subtask:
根據分類支出資料，生成使用 matplotlib 或 plotly 繪製圓餅圖的程式碼。


## 整合圖表到 gradio 介面

### Subtask:
在 `gr.Blocks` 中為財務總結分頁添加一個用於顯示圖表的 Gradio 元件（例如 `gr.Plot` 或 `gr.Image`）。


**Reasoning**:
Add a `gr.Plot` component to the "顯示財務總結" tab in the `gr.Blocks` structure to display the pie chart.



In [ ]:
# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入", id=0):
            # Define inputs for adding expense
            add_date_input = gr.Textbox(label="日期 (YYYY-MM-DD)")
            add_time_input = gr.Textbox(label="時間 (HH:MM)")
            add_item_input = gr.Textbox(label="項目")
            add_amount_input = gr.Textbox(label="金額")
            add_category_input = gr.Radio(['1', '2', '3'], label="分類：1. 收入、2. 吃喝、3. 娛樂")
            # Define a button to trigger the add expense function
            add_expense_button = gr.Button("新增項目")
            # Define the output for the add expense function
            add_expense_output = gr.Textbox(label="結果")

            # Link the button click to the add_expense_ui function
            add_expense_button.click(
                fn=add_expense_ui,
                inputs=[add_date_input, add_time_input, add_item_input, add_amount_input, add_category_input],
                outputs=add_expense_output
            )


        # Create a TabItem for displaying all items
        with gr.TabItem("顯示所有記帳項目", id=1) as display_items_tab:
            # Define the output as a DataFrame to display the items
            display_items_output = gr.DataFrame(label="所有記帳項目 (依時間排序)")

        # Create a TabItem for displaying financial summary
        with gr.TabItem("顯示財務總結", id=2) as display_summary_tab:
             # Define the output as a Textbox to display the summary string
             display_summary_output = gr.Textbox(label="財務總結")
             # Add a component to display the plot
             summary_pie_chart = gr.Plot(label="分類支出圓餅圖")
             # Add a button to trigger displaying the summary and plot
             display_summary_button = gr.Button("顯示財務總結與圖表")


    # Use the .select() method on the tabs object to trigger an action when a tab is selected
    # The index for "顯示所有記帳項目" is 1 (0-indexed)
    tabs.select(
        fn=display_all_items_ui,
        inputs=None, # No inputs needed for display_all_items_ui
        outputs=display_items_output, # The output DataFrame will be updated
        queue=False # Setting queue to False for immediate execution on select
    )

    # Use the .select() method on the tabs object to trigger an action when the summary tab is selected
    # The index for "顯示財務總結" is 2 (0-indexed)
    # We will connect the button click to update both the summary text and the plot in the next step.
    # For now, we just define the components.


# Launch the Blocks app (optional for this step, but good for testing structure)
# demo_blocks.launch(debug=True)

# Task
Create a Gradio application that allows users to add income/expense records to a Google Sheet, display all records from the sheet, and display a financial summary including total income, total expense, balance, and a pie chart of expense categories. The application should have three tabs: "新增支出/收入", "顯示所有記帳項目", and "顯示財務總結". The "顯示財務總結" tab should display the summary text and the pie chart simultaneously.

## 匯入必要的函式庫

### Subtask:
匯入處理資料 (pandas)、時間 (datetime, re)、Google Sheets (gspread, google.colab.auth, google.auth.default) 以及建立使用者介面 (gradio) 所需的函式庫。


**Reasoning**:
Import the necessary libraries for data handling, time processing, Google Sheets interaction, UI creation, and plotting, as specified in the instructions and not already imported in previous cells.



In [ ]:
import pandas as pd
from datetime import datetime
import re
import gspread
from google.colab import auth
from google.auth import default
import gradio as gr
import matplotlib.pyplot as plt
import io
import base64

# Task
在財務總結中加入 Gemini 的分析結果。

## 設定 gemini api

### Subtask:
安裝並設定 Gemini API 的 Python 客戶端。


**Reasoning**:
Install the required library for Gemini API.



# Task
在財務總結欄中加入Gemini的分析

## 修改財務總結函式

### Subtask:
修改 `display_financial_summary` 函式，使其在計算總結後，將總結資料傳送給 Gemini API 進行分析，並將分析結果一併返回。


**Reasoning**:
Modify the `display_financial_summary` function to include the Gemini API call and return the analysis along with the financial data. This involves adding the prompt construction, API call, result extraction, and error handling as described in the instructions.



In [ ]:
def display_financial_summary():
    """Reads data, calculates financial summary, and gets Gemini analysis."""
    try:
        # Access the Google Sheet and read data (similar to display_all_items)
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()

        summary_data = {
            "total_income": 0.00,
            "total_expense": 0.00,
            "total_balance": 0.00,
            "expense_by_category": {},
            "gemini_analysis": "目前沒有足夠的資料進行分析。" # Initialize with a default message
        }

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric and '類型' column exists
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)
            if '類型' not in df.columns:
                df['類型'] = '未知' # Default category if column is missing

            # Calculate total income and expenses
            total_income = df[df['類型'] == '收入']['金額'].sum()
            total_expense = df[df['類型'] != '收入']['金額'].sum()
            total_balance = total_income - total_expense

            # Calculate expenses by category (excluding '收入')
            expense_by_category = df[df['類型'] != '收入'].groupby('類型')['金額'].sum().to_dict()

            summary_data.update({
                "total_income": total_income,
                "total_expense": total_expense,
                "total_balance": total_balance,
                "expense_by_category": expense_by_category
            })

            # --- Add Gemini Analysis ---
            try:
                # Prepare data for Gemini analysis
                analysis_prompt = f"""
                請根據以下的財務資料進行分析並提供建議：

                總收入: {total_income:.2f} 元
                總支出: {total_expense:.2f} 元
                總餘額: {total_balance:.2f} 元

                各分類支出:
                """
                if expense_by_category:
                    for category, amount in expense_by_category.items():
                        analysis_prompt += f"- {category}: {amount:.2f} 元\n"
                else:
                    analysis_prompt += "無分類支出資料。\n"

                analysis_prompt += "\n請提供一個簡潔的財務分析和一些關於支出管理的建議 (請用繁體中文回答)。"

                # Ensure gemini_model is initialized (from a previous cell)
                if 'gemini_model' in globals() and gemini_model:
                    response = gemini_model.generate_content(analysis_prompt)
                    summary_data["gemini_analysis"] = response.text
                else:
                    summary_data["gemini_analysis"] = "Gemini 模型尚未初始化，無法進行分析。"


            except Exception as gemini_e:
                summary_data["gemini_analysis"] = f"呼叫 Gemini API 時發生錯誤: {gemini_e}"
            # --- End of Gemini Analysis ---


        return summary_data

    except Exception as e:
        print(f"計算財務總結時發生錯誤: {e}")
        # Return initial summary_data with error message in analysis
        summary_data["gemini_analysis"] = f"讀取或計算財務資料時發生錯誤: {e}"
        return summary_data

In [12]:
# Install the font package
!apt-get update -qq
!apt-get install -qq ttf-wqy-zenhei

# Clear matplotlib's font cache
import matplotlib.font_manager as fm
fm.fontManager.ttflist.clear()
fm.cachedir = ''
!rm -rf /root/.cache/matplotlib

print("字體安裝完成並清除 Matplotlib 字體快取。請重新執行繪製圖表的儲存格。")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
字體安裝完成並清除 Matplotlib 字體快取。請重新執行繪製圖表的儲存格。


## 更新 gradio 介面

### Subtask:
在「顯示財務總結」分頁中添加一個新的 Gradio 元件，用於顯示 Gemini API 返回的分析結果。


**Reasoning**:
Add a `gr.Textbox` component to the "顯示財務總結" tab in the `gr.Blocks` structure to display the Gemini analysis.



In [ ]:
# Create a new Blocks app instance
with gr.Blocks() as demo_blocks:
    # Create a tabbed container
    with gr.Tabs() as tabs:
        # Create a TabItem for adding expense
        with gr.TabItem("新增支出/收入", id=0):
            # Define inputs for adding expense (using Traditional Chinese labels)
            add_date_input = gr.Textbox(label="請輸入日期 (YYYY-MM-DD)")
            add_time_input = gr.Textbox(label="請輸入時間 (HH:MM)")
            add_item_input = gr.Textbox(label="請輸入項目")
            add_amount_input = gr.Textbox(label="請輸入金額")
            add_category_input = gr.Radio(['1', '2', '3'], label="請選擇分類：1. 收入、2. 吃喝、3. 娛樂")
            # Define a button to trigger the add expense function (using Traditional Chinese label)
            add_expense_button = gr.Button("新增項目")
            # Define the output for the add expense function (using Traditional Chinese label)
            add_expense_output = gr.Textbox(label="結果訊息")

            # Link the button click to the add_expense_ui function
            add_expense_button.click(
                fn=add_expense_ui,
                inputs=[add_date_input, add_time_input, add_item_input, add_amount_input, add_category_input],
                outputs=add_expense_output
            )


        # Create a TabItem for displaying financial summary (using Traditional Chinese label)
        # Update id to 1 as the previous tab is removed
        with gr.TabItem("顯示財務總結", id=1) as display_summary_tab:
             # Define the output as a Textbox to display the summary string (using Traditional Chinese label)
             display_summary_output = gr.Textbox(label="財務總結")
             # Remove the component to display the plot
             # summary_pie_chart = gr.Plot(label="分類支出圓餅圖")
             # Add a component to display the Gemini analysis (using Traditional Chinese label)
             gemini_analysis_output = gr.Textbox(label="Gemini 財務分析與建議")
             # Add a button to trigger displaying the summary and plot (using Traditional Chinese label)
             display_summary_button = gr.Button("顯示財務總結與分析")

             # Link the button click to the display_financial_summary_ui function
             # Update outputs to only expect summary string and Gemini analysis
             display_summary_button.click(
                 fn=display_financial_summary_ui,
                 inputs=None, # No inputs needed
                 outputs=[display_summary_output, gemini_analysis_output] # Output text and Gemini analysis
             )


    # Remove the .select() method for the display_all_items_ui tab
    # Use the .select() method on the tabs object to trigger an action when a tab is selected
    # The index for "顯示所有記帳項目" was 1 (0-indexed)
    # tabs.select(
    #     fn=display_all_items_ui,
    #     inputs=None, # No inputs needed for display_all_items_ui
    #     outputs=display_items_output, # The output DataFrame will be updated
    #     queue=False # Setting queue to False for immediate execution on select
    # )

    # Use the .select() method on the tabs object to trigger an action when the summary tab is selected
    # The index for "顯示財務總結" is now 1 (0-indexed)
    # We will connect the button click to update both the summary text and the plot in the next step.
    # For now, we just define the components.


# Launch the Blocks app
demo_blocks.launch(debug=True)

# Modify the display_financial_summary_ui function to only return two values
def display_financial_summary_ui():
    """Calls the financial summary function and prepares data for UI display."""
    summary_data = display_financial_summary()

    # Format the summary data into a string for the Textbox output
    summary_string = f"財務總結:\n"
    summary_string += f"總收入: {summary_data['total_income']:.2f} 元\n"
    summary_string += f"總支出: {summary_data['total_expense']:.2f} 元\n"
    summary_string += f"總餘額: {summary_data['total_balance']:.2f} 元\n\n" # Corrected variable name

    summary_string += "各分類支出:\n"
    if summary_data['expense_by_category']:
        for category, amount in summary_data['expense_by_category'].items():
            summary_string += f"{category}: {amount:.2f} 元\n"
    else:
        summary_string += "無分類支出資料。"

    # Return the summary string and the Gemini analysis
    return summary_string, summary_data.get("gemini_analysis", "無分析結果。")

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://98a6796a54786e5c13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^